In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
import os
load_dotenv()
PROJECT_ROOT = os.getenv('PROJECT_ROOT')

# Add PROJECT_ROOT to the Python path
import sys
sys.path.append(PROJECT_ROOT)
from planner2 import mcmc_step



In [2]:
import pandas as pd

ORIGIN_ICAO = 'EGLL'
DEST_ICAO = 'UKBB'
ORIGIN_RWY = '09R'
DEST_RWY = '18R'

airports_df = pd.read_csv(os.path.join(PROJECT_ROOT, "data", "airac", "airports.csv"))


# Get the latitude and longitude of the origin and destination
origin_lat = airports_df[airports_df['icao'] == ORIGIN_ICAO]['latitude'].values[0]
origin_lon = airports_df[airports_df['icao'] == ORIGIN_ICAO]['longitude'].values[0]
dest_lat = airports_df[airports_df['icao'] == DEST_ICAO]['latitude'].values[0]
dest_lon = airports_df[airports_df['icao'] == DEST_ICAO]['longitude'].values[0]
# Origin and destination airport names
origin_name = airports_df[airports_df['icao'] == ORIGIN_ICAO]['name'].values[0]
dest_name = airports_df[airports_df['icao'] == DEST_ICAO]['name'].values[0]

print(f'Origin: {ORIGIN_ICAO} - {origin_name} ({origin_lat}, {origin_lon}, {ORIGIN_RWY})')
print(f'Destination: {DEST_ICAO} - {dest_name} ({dest_lat}, {dest_lon}, {DEST_RWY})')

Origin: EGLL - HEATHROW (51.4775, -0.461389, 09R)
Destination: UKBB - BORYSPIL INTL (50.344722, 30.893333, 18R)


In [3]:
from nav_graph import generate_navigraph

route_graph = generate_navigraph(ORIGIN_ICAO, DEST_ICAO, origin_lat, origin_lon, dest_lat, dest_lon,
                                  ORIGIN_RWY, DEST_RWY,
                                  w_dct=1.0, w_fra=1.0, w_proc=0.2)


Subsetting the ATS graph to the great circle path between origin and destination...


Adding edges to subset: 100%|██████████| 25011/25011 [00:00<00:00, 809579.85it/s]


ATS graph loaded. Nodes: 1670, edges: 3817
Building FRA routing options...
Found 1037 FRA points within 100nm of the great circle path between origin and destination.          
Merging these into the ATS graph...
223 FRA points renamed for BALTIC & FRAIT & SECSI & SEE FRA


BALTIC & FRAIT & SECSI & SEE FRA: 100%|██████████| 267/267 [00:00<00:00, 525.57it/s]


35 FRA points renamed for BELFRA


...

127 FRA points renamed for UKNESFRA


UKNESFRA: 100%|██████████| 133/133 [00:00<00:00, 821.41it/s]


FRA graph merged into ATS graph. Nodes: 2667, edges: 121054
Computing cost for the ATS-FRA route graph...


Adding edges to subset: 100%|██████████| 121054/121054 [00:00<00:00, 312216.62it/s]


Adding SID and STAR graphs to the subset...
Route graph subset created. Nodes: 2923, edges: 121334
Great circle distance between origin and destination: 1179.84 nm


In [4]:
import networkx as nx
from utils.flightplans import format_flightplan, get_detailed_flightplan_from_waypoint_list
from utils.haversine import haversine_distance

# Find the shortest path between origin and destination
origin_node = f"{ORIGIN_ICAO}_{ORIGIN_RWY}"
dest_node = f"{DEST_ICAO}_{DEST_RWY}"

# Check if the nodes exist in the graph
if origin_node not in route_graph.nodes:
    print(f"Origin node {origin_node} not found in graph. Available nodes for {ORIGIN_ICAO}:")
    for node in route_graph.nodes:
        if node.startswith(ORIGIN_ICAO):
            print(f"  {node}")
    # Try to find an alternative
    for node in route_graph.nodes:
        if node.startswith(ORIGIN_ICAO):
            origin_node = node
            print(f"Using {origin_node} as origin node instead")
            break

if dest_node not in route_graph.nodes:
    print(f"Destination node {dest_node} not found in graph. Available nodes for {DEST_ICAO}:")
    for node in route_graph.nodes:
        if node.startswith(DEST_ICAO):
            print(f"  {node}")
    # Try to find an alternative
    for node in route_graph.nodes:
        if node.startswith(DEST_ICAO):
            dest_node = node
            print(f"Using {dest_node} as destination node instead")
            break

# Find the shortest path
try:
    shortest_path = nx.shortest_path(route_graph, source=origin_node, target=dest_node, weight='cost')
    print(f"Shortest path found with {len(shortest_path)} waypoints")
    
    
    result = get_detailed_flightplan_from_waypoint_list(route_graph, shortest_path)

    # Print the flight plan
    print(format_flightplan(result))

except nx.NetworkXNoPath:
    print(f"No path found between {origin_node} and {dest_node}")
except Exception as e:
    print(f"Error finding path: {e}")

Origin node EGLL_09R not found in graph. Available nodes for EGLL:
  EGLL
Using EGLL as origin node instead
Destination node UKBB_18R not found in graph. Available nodes for UKBB:
  UKBB
Using UKBB as destination node instead
Shortest path found with 26 waypoints
EGLL CPT5J WOD BPK Q295 BRAIN M197 REDFA DERAM L980 POLON M70 OKROT SLV SLV2J UKBB


# MCMC Sampling with Metropolis-Hastings

In [22]:
MAX_ITER = 100_000
BURN_IN = 5_000 # can go as high as 10_000
THINNING = 25 # can go as high as 50 
sampled_routes = []
temperature = 1000

route = shortest_path

for i in range(MAX_ITER):
    route, accepted = mcmc_step(route_graph, route, temperature)
    if i > BURN_IN and i % THINNING == 0:
        sampled_routes.append(route)
    print(f"{'Accepted' if accepted else 'Rejected'} route in iteration {i}")



Time taken to sample a and c: 0.40798187255859375
Time taken to find the pivot node: 18.777167558670044
Time taken to sample the pivot node: 0.0012013912200927734
Time taken to propose the new route: 0.0005903244018554688
Time taken to evaluate the cost of the new route: 0.0
Time taken to compute the acceptance probability: 0.0
Rejected route in iteration 0
Time taken to sample a and c: 0.4144771099090576
Time taken to find the pivot node: 30.5644850730896
Time taken to sample the pivot node: 0.0
Time taken to propose the new route: 0.0009989738464355469
Time taken to evaluate the cost of the new route: 0.0
Time taken to compute the acceptance probability: 0.0
Accepted route in iteration 1
Time taken to sample a and c: 0.4142141342163086
Time taken to find the pivot node: 28.16599130630493
Time taken to sample the pivot node: 0.0010001659393310547
Time taken to propose the new route: 0.0011599063873291016
Time taken to evaluate the cost of the new route: 0.0
Time taken to compute the a

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'